<a href="https://colab.research.google.com/github/PatrykCiwinski/NLP_sentiment_transformers/blob/main/NLP_sentiment_transformers_Provident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web-scrapping

In [211]:
import requests
from bs4 import BeautifulSoup

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
}
URL = "https://www.zadluzenia.com/provident-opinie/"
page = requests.get(URL,headers=header)

soup = BeautifulSoup(page.content, "html.parser")

In [219]:
reviews= soup.select('div#comments')

In [233]:
all_comments = []
for page_number in range(1, 5):
  URL = f"https://www.zadluzenia.com/provident-opinie/comment-page-{page_number}"
  page = requests.get(URL,headers=header)
  soup = BeautifulSoup(page.content, "html.parser")
  reviews= soup.select('div#comments')
    
  for review in reviews:
          inner_div = soup.select("div.comments")
          for text in inner_div:
            comments = text.find_all("p")
            for comment in comments:
              all_comments.append(comment.text.strip())

In [235]:
len(all_comments)

12

##Text sentiment using transformers

In [ ]:
!pip install transformers

In [ ]:
!pip install xformers

In [278]:
import pandas as pd

In [253]:
import tensorflow as tf
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification
import numpy as np

In [ ]:
model_name="bardsai/twitter-sentiment-pl-base"

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [265]:
# clf = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)

In [266]:
# clf('nie wiem gdzie ja miałem oczy.Tak sobie')

In [270]:
X=all_comments

In [271]:
batch = tokenizer(X,padding=True,truncation=True,max_length=512,return_tensors='tf')

In [272]:
outputs = model(**batch)

In [273]:
predictions = tf.nn.softmax(
    outputs.logits, axis=1)

In [274]:
preds=np.argmax(predictions, axis=1)

In [275]:
preds

array([2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1])

In [276]:
labels =[model.config.id2label[id] for id in preds]

In [306]:
df = pd.DataFrame({
    'Opinia': X,
    'Sentyment': labels
},index=np.arange(len(X))+1)

In [307]:
df

,Opinia,Sentyment
1,przelałem spłatę pożyczki 4 dni temu wciąż bra...,negative
2,Pierwszy i ostatni raz w życiu skorzystałam z ...,negative
3,Największa lichwa w RP.\r\nZlodzieje,negative
4,"Oferty są bardzo kuszące, niski procent, nawet...",positive
5,"firma okradli mnie ,albo ich przedstawiciel na...",negative
6,Nigdy nie polecę! Nie bierzcie stąd pożyczek b...,negative
7,Od wielu lat korzystam z pożyczek w Prowidenc...,positive
8,Problem ze zwrotem nadpłaty od spłaconej pożyc...,negative
9,Banda złodzieji! Biorąc pożyczkę licz się z ty...,negative
10,Jak dla mnie złodziejstwo. Ta ich tabelka ze s...,negative


In [308]:
df['Sentyment']=df['Sentyment'].map({'negative':'negatywny','positive':'pozytywny','neutral':'neutralny'})

In [309]:
df

,Opinia,Sentyment
1,przelałem spłatę pożyczki 4 dni temu wciąż bra...,negatywny
2,Pierwszy i ostatni raz w życiu skorzystałam z ...,negatywny
3,Największa lichwa w RP.\r\nZlodzieje,negatywny
4,"Oferty są bardzo kuszące, niski procent, nawet...",pozytywny
5,"firma okradli mnie ,albo ich przedstawiciel na...",negatywny
6,Nigdy nie polecę! Nie bierzcie stąd pożyczek b...,negatywny
7,Od wielu lat korzystam z pożyczek w Prowidenc...,pozytywny
8,Problem ze zwrotem nadpłaty od spłaconej pożyc...,negatywny
9,Banda złodzieji! Biorąc pożyczkę licz się z ty...,negatywny
10,Jak dla mnie złodziejstwo. Ta ich tabelka ze s...,negatywny
